In [ ]:
!pip install sentence-transformers
!pip install faiss-gpu

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pickle
import faiss


# Get Content Embeddings

In [ ]:
# content_path = data break 430.csv

content_path = 'data_break430_231111.csv'
df = pd.read_csv(content_path)
df.head(3)

,Unnamed: 0,Url,Combined_Content_2
0,0,alzheimer,What is Alzheimer's disease? Alzheimer's is a ...
1,1,alzheimer,"Reason. Therefore, the brain is considered a g..."
2,2,alzheimer,Common symptoms of Alzheimer's syndrome. The s...


In [ ]:
model_name = 'sentence-transformers/all-mpnet-base-v2'
model = SentenceTransformer(model_name)

content_list = df['Combined_Content_2'].to_list()
embedding_list = model.encode(content_list, show_progress_bar=True)

In [ ]:
df['Embeddings'] = pd.Series(list(embedding_list))
df = df.rename(columns={'Combined_Content_2': 'Content'})
df.head(3)

# Get Faiss index

In [ ]:
dimension = 768
index = faiss.IndexFlatL2(dimension)

embeddings_stack = np.vstack(df['Embeddings'].values)
index.add(embeddings_stack)

In [ ]:
# Save the index to the specified file
index_file_path = 'faiss_768_break.index'
faiss.write_index(index, index_file_path)

In [ ]:
# Create csv file to save Content
df = df.drop(columns='Embeddings')
df.to_csv('df_768_break.csv', index=False)